# dissipation

## Setup

In [ ]:
# If necessary:
import os
os.remove('snorkel.db')

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

## Parsing

In [ ]:
from snorkel import SnorkelSession
session = SnorkelSession()

### Define Parser

In [ ]:
import os
from snorkel.parser import CorpusParser, HTMLParser, OmniParser
from snorkel.utils import get_ORM_instance
from snorkel.queries import split_corpus

docs_path = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware100_html/'
doc_parser = HTMLParser(path=docs_path)
context_parser = OmniParser()
cp = CorpusParser(doc_parser, context_parser, max_docs=88) # Save 10 for final evaluation 

### Run Parser

In [ ]:
%time corpus = cp.parse_corpus(name='Hardware', session=session)

session.add(corpus)
session.commit()

### Split Corpus

In [ ]:
from snorkel.models import Corpus

corpus = get_ORM_instance(Corpus, session, 'Hardware')
split_corpus(session, corpus, train=0.8, development=0.2, test=0, seed=3)

In [ ]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ corpus');

## Extraction

In [1]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ corpus snorkel.db');

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

from snorkel import SnorkelSession
session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

Part_Dissipation = candidate_subclass('Part_Dissipation', ['part','dissipation'])

### Define Matchers

In [3]:
from snorkel.matchers import RegexMatchSpan, Union

eeca_matcher = RegexMatchSpan(rgx='([b]{1}[abcdefklnpqruyz]{1}[\swxyz]?[0-9]{3,5}[\s]?[A-Z\/]{0,5}[0-9]?[A-Z]?([-][A-Z0-9]{1,7})?([-][A-Z0-9]{1,2})?)')
jedec_matcher = RegexMatchSpan(rgx='([123]N\d{3,4}[A-Z]{0,5}[0-9]?[A-Z]?)')
jis_matcher = RegexMatchSpan(rgx='(2S[abcdefghjkmqrstvz]{1}[\d]{2,4})')
others_matcher = RegexMatchSpan(rgx='((NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|TIS|TIPL|DTC|MMBT|PZT){1}[\d]{2,4}[A-Z]{0,3}([-][A-Z0-9]{0,6})?([-][A-Z0-9]{0,1})?)')
parts_matcher = Union(eeca_matcher, jedec_matcher, jis_matcher, others_matcher)

dissipation_matcher = RegexMatchSpan(rgx=r'\d\d[05]')

### Define ContextSpaces

In [4]:
from snorkel.candidates import OmniNgrams
from hardware_utils import OmniNgramsPart, get_gold_dict
from collections import defaultdict
import os

# Make parts list
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
gold_parts = get_gold_dict(gold_file, doc_on=True, part_on=True, val_on=False)
parts_by_doc = defaultdict(set)
for part in gold_parts:
    parts_by_doc[part[0]].add(part[1])

# Baseline 1: all candidates, no splitting/linking, majority label
part_ngrams        = OmniNgrams(n_max=1)
dissipation_ngrams = OmniNgrams(n_max=1)

# Baseline 2: candidates w/ throttler, no splitting/linking, majority label

# Current 1: candidates (w/ throttler?), splitting, no linking, supervised learning

# Oracle 1: candidates (w/ throttler?), splitting/linking, supervised learning

# part_ngrams = OmniNgramsPart(n_max=2, parts_by_doc=None) # NOTE: linking is turned off


### Run CandidateExtractor

In [5]:
from snorkel.models import Corpus
from snorkel.candidates import CandidateExtractor
from snorkel.utils import get_ORM_instance
ce = CandidateExtractor(Part_Dissipation, 
                        [part_ngrams, dissipation_ngrams], 
                        [parts_matcher, dissipation_matcher], 
                        throttler=None)

for corpus_name in ['Hardware Training', 'Hardware Development']:
    corpus = get_ORM_instance(Corpus, session, corpus_name)
    print "Extracting Candidates from %s" % corpus
    %time candidates = ce.extract(\
        corpus.documents, corpus_name + ' Candidates', session)
    session.add(candidates)
    print "%s contains %d Candidates" % (candidates, len(candidates))
session.commit()

Extracting Candidates from Corpus (Hardware Training)
[========================================] 100%
CPU times: user 2min 16s, sys: 1.47 s, total: 2min 17s
Wall time: 2min 20s
Candidate Set (Hardware Training Candidates) contains 132014 Candidates
Extracting Candidates from Corpus (Hardware Development)
[========================================] 100%
CPU times: user 17.9 s, sys: 249 ms, total: 18.1 s
Wall time: 19.4 s
Candidate Set (Hardware Development Candidates) contains 15064 Candidates


### Assess Recall

In [19]:
# MAJORITY LABEL
from snorkel.models import CandidateSet
from snorkel.utils import get_ORM_instance
from hardware_utils import candidates_to_entities, count_labels

# map train candidates to entities
train_candidates = get_ORM_instance(CandidateSet, session, 'Hardware Training Candidates')
train_entities   = candidates_to_entities(train_candidates)

# get majority label for training set
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
gold_entities = get_gold_dict(gold_file, attrib='dev_dissipation')
T, F = count_labels()




[========================================] 100%


In [25]:
from hardware_utils import count_labels

# print len(train_candidates)
# print len(train_entities)
# print list(train_entities)[0]
# get majority label
# (tp, fp, fn) = entity_level_total_recall(
#     train, gold_file, 'dev_dissipation', relation=True)

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
gold_entities = get_gold_dict(gold_file, attrib='dev_dissipation')
T, F = count_labels(train_entities, gold_entities)
print T, F

467 15401


In [ ]:
# find majority label for training set
train = get_ORM_instance(CandidateSet, session, 'Hardware Training Candidates')
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(tp, fp, fn) = entity_level_total_recall(
    train, gold_file, 'dev_dissipation', relation=True)

In [ ]:
all_candidates = session.query(Candidate).all()

In [ ]:
from snorkel.models import Candidate
from hardware_utils import entity_level_total_recall

all_candidates = session.query(Candidate).all()
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(tp, fp, fn) = entity_level_total_recall(
    all_candidates, gold_file, 'dev_dissipation', relation=True)

In [ ]:
from pprint import pprint
pprint(fn)
print len(fn)

In [ ]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ candidates');

## Gold Labels

In [ ]:
# If necessary
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ candidates snorkel.db');

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
Part_Dissipation = candidate_subclass('Part_Dissipation', ['part','dissipation'])

In [ ]:
import os
from snorkel.models import CandidateSet
from hardware_utils import load_hardware_labels

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
for set_name in ['Training', 'Development']:
    candidate_set_name = 'Hardware %s Candidates' % set_name
    candidates = session.query(CandidateSet).filter(
        CandidateSet.name == candidate_set_name).one()
    label_set_name = 'Hardware %s Candidates -- Gold' % set_name
    annotation_key_name = 'Hardware %s Labels -- Gold' % set_name
    %time gold_candidates, annotation_key = load_hardware_labels(session,\
                           label_set_name, \
                           annotation_key_name, \
                           candidates, \
                           gold_file, \
                           'dissipation')
    candidates_gold = session.query(CandidateSet).filter(
        CandidateSet.name == candidate_set_name + ' -- Gold').one()
    print "%d/%d Candidates in %s have positive Labels" % (
        len(candidates_gold), len(candidates), candidates)

In [ ]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ labels');

### Assess Baseline Performance

In [ ]:
from snorkel.models import Corpus, Candidate
from snorkel.utils import get_ORM_instance
from snorkel.lf_helpers import *
from hardware_utils import baseline, entity_level_f1
from itertools import chain

def LF_all_true(c):
    return True

def LF_complement_nearby(c):
    return 1 if set(['complement','complementary']).isdisjoint(
        chain.from_iterable(
            [get_phrase_ngrams(c.part), 
             get_phrase_ngrams(c.dissipation),
             get_neighbor_phrase_ngrams(c.dissipation)])) else 0

candidates = session.query(Candidate).all()
(tp, fp, tn, fn) = baseline(candidates, LF_complement_nearby)

corpus = get_ORM_instance(Corpus, session, 'Hardware')
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(TP, FP, FN) = entity_level_f1(tp, fp, tn, fn, gold_file, corpus, 'dissipation')

In [ ]:
from pprint import pprint
pprint(FP)

In [ ]:
pprint(FN)

In [ ]:
from hardware_utils import entity_to_candidates

print len(candidates)
matches = entity_to_candidates((u'SIEMS01215-1', u'BC856', u'NPN'), candidates)
c = matches[0]
print c

In [ ]:
from snorkel.models import ImplicitSpan
from snorkel.lf_helpers import *
phrases = []
print list(get_neighbor_phrase_ngrams(c.dissipation))
print c.dissipation.parent
print phrases
print "SPAN_TYPE_[%s]" % ('IMPLICIT' if isinstance(c.dissipation, ImplicitSpan) else 'EXPLICIT')
# list(get_phrase_ngrams(c.dissipation))

## Features

In [ ]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ labels snorkel.db');

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
Part_Dissipation = candidate_subclass('Part_Dissipation', ['part','dissipation'])

### Extract Features

In [ ]:
from snorkel.models import CandidateSet
from snorkel.fast_annotations import FeatureManager
from snorkel.utils import get_ORM_instance

train = get_ORM_instance(CandidateSet, session, 'Hardware Training Candidates')
dev   = get_ORM_instance(CandidateSet, session, 'Hardware Development Candidates')

feature_manager = FeatureManager()
%time F_train = feature_manager.create(session, train, 'Train Features')
%time F_dev = feature_manager.update(session, dev, 'Train Features', expand_key_set=False)

In [ ]:
# If necessary:
import os
os.system('cp snorkel.db snorkel.db\ featurized');

## LFs

In [ ]:
# If necessary
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ featurized snorkel.db');

# from snorkel import SnorkelSession
# session = SnorkelSession()

# import sys
# sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

# from snorkel.models import candidate_subclass
# Part_Dissipation = candidate_subclass('Part_Dissipation', ['part','dissipation'])

# from snorkel.models import CandidateSet
# train = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates').one()

### Define LFs

In [ ]:
from snorkel.fast_annotations import LabelManager
from snorkel.lf_helpers import *
import re
label_manager = LabelManager()

LFs = []

def LF_complement_phrase_part(c):
    return -1 if overlap(['complement','complementary'], 
                         get_phrase_ngrams(c.part)) else 0
LFs.append(LF_complement_phrase_part)

def LF_complement_phrase_dissipation(c):
    return -1 if overlap(['complement','complementary'], 
                         get_phrase_ngrams(c.dissipation)) else 0
LFs.append(LF_complement_phrase_dissipation)

def LF_complement_neighbor_dissipation(c):
    return -1 if overlap(['complement','complementary'], 
                         get_neighbor_phrase_ngrams(c.dissipation)) else 0
LFs.append(LF_complement_neighbor_dissipation)

def LF_top_mark_col_part(c):
    return -1 if overlap(['top','mark'],
                         get_col_ngrams(c.part)) else 0
LFs.append(LF_top_mark_col_part)

def LF_endswith_D_part(c):
    return -1 if c.part.get_span().endswith('D') else 0
LFs.append(LF_endswith_D_part)

def LF_default_positive(c):
    return 1 if not overlap(['complement','complementary'],
                            chain.from_iterable([
                            get_phrase_ngrams(c.part),
                            get_phrase_ngrams(c.dissipation),
                            get_neighbor_phrase_ngrams(c.dissipation)])) else 0
LFs.append(LF_default_positive)

In [ ]:
from snorkel.models import Candidate
from hardware_utils import entity_to_candidates

candidates = session.query(Candidate).all()
print len(candidates)
# matches = entity_to_candidates((u'SIEMS01215-1', u'BC856', u'NPN'), candidates)
# c = matches[0]
c = candidates[0]
print candidates[0]

print LF_complement_phrase_part(c)
print LF_complement_phrase_dissipation(c)
print LF_complement_neighbor_dissipation(c)
print LF_top_mark_col_part(c)
print LF_endswith_D_part(c)
print LF_default_positive(c)

### Apply LFs

In [ ]:
%time L_train = label_manager.create(session, train, 'LF Labels', f=LFs)
L_train

### Assess LF accuracy

In [ ]:
train_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates -- Gold').one()
%time L_train.lf_stats(train_gold)

In [ ]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ features');

## Learn and Evaluate

In [ ]:
# If necessary:
# import os
# os.remove('snorkel.db');
# os.system('cp snorkel.db\ features snorkel.db');

# from snorkel import SnorkelSession
# session = SnorkelSession()

# import sys
# sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

# from snorkel.models import candidate_subclass
# Part_Dissipation = candidate_subclass('Part_Dissipation', ['part','dissipation'])

# from snorkel.models import CandidateSet
# train = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Training Candidates').one()
# dev = session.query(CandidateSet).filter(
#     CandidateSet.name == 'Hardware Development Candidates').one()

# from snorkel.annotations import FeatureManager, LabelManager
# feature_manager = FeatureManager()
# %time F_train = feature_manager.load(session, train, 'Train Features')
# %time F_dev = feature_manager.load(session, dev, 'Train Features')

# label_manager = LabelManager()
# %time L_train = label_manager.load(session, train, 'LF Labels')

In [ ]:
from snorkel.learning import NaiveBayes

gen_model = NaiveBayes()
gen_model.train(L_train, n_iter=15000, rate=1e-2)
%time gen_model.save(session, 'Generative Params')
train_marginals = gen_model.marginals(L_train)

In [ ]:
from snorkel.learning import LogReg

disc_model = LogReg()
disc_model.train(F_train, train_marginals, n_iter=2000, rate=1e-5)
%time disc_model.save(session, "Discriminative Params")

In [ ]:
train_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates -- Gold').one()

In [ ]:
dev_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Development Candidates -- Gold').one()

In [ ]:
from snorkel.models import CandidateSet
from snorkel.annotations import LabelManager
label_manager = LabelManager()
L_dev = label_manager.load(session, dev, 'Hardware Development Labels -- Gold')

In [ ]:
tp, fp, tn, fn = disc_model.score(F_dev, L_dev, dev_gold)

In [ ]:
from snorkel.models import Corpus
from hardware_utils import entity_level_f1
import os

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Development').one()
(TP, FP, FN) = entity_level_f1(tp, fp, tn, fn, gold_file, corpus, 'dissipation')

In [ ]:
from pprint import pprint
FP_list = sorted(list(FP))
pprint(FP_list)

In [ ]:
from hardware_utils import entity_to_candidates

for i in range(1):
    entity = FP_list[i]
    print entity
    print

    matches = entity_to_candidates(entity, fp)
    # print "# Matches: %d" % len(matches)
    candidate = matches[0]
    # print candidate
    print

    print disc_model.get_candidate_score(candidate, F_dev)
    print
    pprint(disc_model.get_candidate_feature_weights(candidate, F_dev))

In [ ]:
for ngram in candidate.part.get_attrib_tokens():
    print "CONTAINS_%s_[%s]" % ('words'.upper(), ngram)

In [ ]:
from hardware_utils import part_error_analysis
part_error_analysis(candidate)

In [ ]:
import os
os.system('cp snorkel.db snorkel.db\ final');